In [17]:
import numpy as np 
import pandas as pd 

                                                        Методы решения уравнений переноса
Найти аналитическое и численное решение смешанной задачи для уравнения переноса в квадрате $0 \le x, t \le 1$ и сравнить их значения в одиннадцати 
равноудаленных точках в момент времени $t = 1$ 



      Задание 2.
Дифференциальная задача:
\begin{equation*}
 \begin{cases}
    \frac{du}{dt} + 4\frac{du}{dx} = x, 0 < t \le 1, 0 \le x < 1 \\
    u(x, 0) = sin(x) + 0.125x^{2}, u(0, t) = -sin(4t)     
 \end{cases}
\end{equation*}
Разностная схема:
$$D_h = {(x_l, t^n) : x_l = lh, hL = 1, l = 0,...,L; t^n = n \tau, \tau N = 1, n = 0,..., N}$$

\begin{equation*}
 \begin{cases}
      u_l^{n+1} = u_l^n + \frac{2\tau}{3h}(2u_{l-3}^n - 9u_{l-2}^n + 18u_{l-1}^n - 11u_l^n) + \\
                + \frac{8\tau^2}{h^2}(-u_{l-3}^n + 4u_{l-2}^n - 5u_{l-1}^n + 2u_l^n) - \\
                - \frac{32\tau^3}{3h^3}(-u_{l-3}^n + 3u_{l-2}^n - 3u_{l-1}^n + u_l^n) + \tau x_l - 2\tau^{2}, l = 3,...,L; n = 0,...,N-1 \\
 \end{cases}
\end{equation*}


        Аналитическое решение 
Первые интеграллы:
$$dt = \frac{dx}{4}$$
$$\frac{du}{x} = \frac{dx}{4}$$
Следовательно их решения:
$$v_1 = x - 4t$$
$$v_2 = 0.125x^2 - u$$
Соответственно в $t = 0$:
$$x = v_1$$
$$u = 0.125x^2 - v_2$$
Подставив $x, u$ в первое начальное условие получаем:
$$u = sin(x - 4t) + 0.125(x - 4t)^2$$
Соответственно в $x = 0$:
$$t = -v_1 / 4$$
$$u = -v_2$$
Подставив $t, u$ в первое начальное условие получаем:
$$u = sin(x - 4t) + 0.125x^2$$
Тогда аналитическое решение выглядит, вот так:
$$u = sin(x - 4t) + 0.125x^2$$

In [18]:

def analitical_solution(x, t):
    return np.sin(x - 4 * t) + 0.125 * x**2

        Численное решение

In [19]:
def grid(L, N):
    '''
    returns np.arrays of x, t broken into L and N pieces
    '''
    return np.linspace(0, 1, L), np.linspace(0, 1, N)

#начальные условия
def phi(x):
    return np.sin(x) + 0.125 * x**2

def psi(t):
    return -np.sin(4 * t)

def u_x(t):
    return np.cos(4 * t)

def u_xx(t):
    return np.sin(4 * t) + 1/4

def u_xxx(t):
    return -np.cos(4 * t)

def numerical_solve(x_n, t_n, L, M, k):
    h = 1 / (L - 1)
    tau = h * k
    u = [x[:] for x in [[0] * L] * M]
    u[0] = phi(x_n)
    psi_m = psi(t_n)
    u_x_m = u_x(t_n)
    u_xx_m = u_xx(t_n)
    u_xxx_m = u_xxx(t_n)
    for m in range(1, M):
        u[m][0] = psi_m[m]
        u[m][1] = psi_m[m] + u_x_m[m] * h + u_xx_m[m] * h**2 / 2 + u_xxx_m[m] * h**3 / 6
        u[m][2] = psi_m[m] + u_x_m[m] * 2 * h + u_xx_m[m] * h**2 * 2 + u_xxx_m[m] * (h**3) * 4 / 3
    for m in range(M - 1):
        for l in range(3, L):
            u[m + 1][l] = u[m][l] + (2 * u[m][l - 3] - 9 * u[m][l - 2] + 18 * u[m][l - 1] - 11 * u[m][l]) * 2 * tau / (3 * h) + (-u[m][l - 3] + 4 * u[m][l - 2] - 5 * u[m][l - 1] + 2 * u[m][l]) * 8 * (tau**2) / (h**2) - (-u[m][l - 3] + 3 * u[m][l - 2] - 3 * u[m][l - 1] + u[m][l]) * 32 * (tau**3) / (3 * h**3) + tau * x_n[l] - 2 * tau**2
            
    return u

        Вывод решения

In [20]:
#находим аналитическое решение
x_rep = np.linspace(0, 1, 11)
u_analytical = analitical_solution(x_rep, 1)

#находим численное рашение
k = 0.2 
L = 11
M = int((L - 1) / k) + 1
x, t = grid(L, M)
u_n = numerical_solve(x, t, L, M, k)
u_numeric = np.array(u_n[-1][::int((len(u_n[0]) - 1) / 10)])

# запись данных в словарь и передча его в DataFrame для красоты
dat = {
    'x' : x_rep,
    'Analytical solve' : u_analytical,
    'Numeric solve' : u_numeric,
    'Error' : np.abs(u_analytical - u_numeric),
    'Max error' : np.max(np.abs(u_analytical - u_numeric))
}
df = pd.DataFrame(data=dat)
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10
x,0.000000,0.100000,0.200000,0.300000,0.400000,0.500000,0.600000,0.700000,0.800000,0.900000,1.000000
Analytical solve,0.756802,0.689016,0.616858,0.541086,0.462520,0.382033,0.300541,0.218996,0.138374,0.059669,-0.016120
Numeric solve,0.756802,0.689013,0.616809,0.541036,0.462478,0.381998,0.300513,0.218974,0.138359,0.059660,-0.016123
Error,0.000000,0.000003,0.000049,0.000050,0.000042,0.000035,0.000028,0.000022,0.000015,0.000009,0.000003
Max error,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050
